## Day Trip Planner

My day trip planner will help plan my day with a friend. We plan to meet in between our houses for coffee and go to a kids park.

Kaggles - "Agents Intensive - Capstone Project"

Problem Statement -- Me and my friend decide to plan a play day together with kids. First we plan to go to a coffee shop and then go to a children's park to spend rest of the day. We both decide to meet at a location between our two houses where there are best coffee shops and a good children's play park.

The Day Trip Planner agent will help us plan our day wisely.
Why agents? -- Why are agents the right solution to this problem

The agent will help find the best coffee shops and parks based on the reviews and in between the 2 locations we want. It will help save our time finding them manually through each and checking their reviews between the two locations. 

### Setup

1. Setup the Google Gemini API key
2. Setup the Google Map Api key

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    GOOGLE_MAPS_API_KEY = UserSecretsClient().get_secret("GOOGLE_MAPS_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_MAPS_API_KEY"] = GOOGLE_MAPS_API_KEY
    print("✅ Gemini API key and Google Map key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' and 'GOOGLE_MAPS_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key and Google Map key setup complete.


In [2]:
# Setup the environment with the needed imports

import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.adk.runners import Runner

APP_NAME = "My_Day_Planner"  # Application
USER_ID = "sami"  # User
SESSION = "default"  # Session
MODEL_NAME = "gemini-2.5-flash"

#from google.adk.apps.app import App, ResumabilityConfig
#from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")



✅ ADK components imported successfully.



Helper functions
=================

Helper function that manages a complete conversation session, handling session creation/retrieval, query processing, and response streaming. It supports both single queries and multiple queries in sequence.

In [3]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.



Configure Retry Options
=======================

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

Create the MCP Toolset
======================

The McpToolset is used to integrate an ADK Agent with an MCP Server.
This McpToolSet will use the Google map api's to find a place based on user queries

In [5]:

# MCP integration with Google maps Server
mcp_map_server = McpToolset(
        connection_params=StdioConnectionParams(
        server_params = StdioServerParameters(
            command='npx',
            args=[
                "-y",
                "@modelcontextprotocol/server-google-maps",
            ],
            # Pass the API key as an environment variable to the npx process
            # This is how the MCP server for Google Maps expects the key.
            env={
                "GOOGLE_MAPS_API_KEY": GOOGLE_MAPS_API_KEY
            }
        ),
        timeout=30,
    ),
    # You can filter for specific Maps tools if needed:
    tool_filter=['maps_geocode', 'maps_search_places']
)

print("✅ MCP Tool created")



✅ MCP Tool created


Create Coffee agent with MCP integration
========================================

This agent will find a good coffee shop between 2 given locations

In [6]:

coffee_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="GoogleMap_agent",
    description='A helpful assistant for user questions related to locations,maps & directions',
    instruction="""Answer user questions to the best of your knowledge Your only job is to use the
    mcp_map_server tool to find 2 pieces of relevant information on the given topic and present the findings """,
    tools=[mcp_map_server],
    output_key="coffeeLocation_findings",  # The result of this agent will be stored in the session state with this key.
)

Create Park agent with MCP integration
=======================================

This agent will find a good Childrens Park between 2 given locations


In [7]:
park_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="GooglePark_agent",
    description='A helpful assistant for user questions related to locations,maps & directions',
    instruction="""Answer user questions to the best of your knowledge Your only job is to use the
    mcp_map_server tool to find 2 pieces of relevant information on the given topic and present the findings """,
    tools=[mcp_map_server],
    output_key="parkLocation_findings",  # The result of this agent will be stored in the session state with this key.
)

The AggregatorAgent 
===================

This agent runs *after* the parallel step to synthesize the results.
This agent will aggregate the results of Coffee agent and Park agent and summarize the results

In [8]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
# This agent will aggregate the results of Coffee agent and Park agent

aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these two findings into a single summary:

    **Coffee place nearby :**
    {coffeeLocation_findings}
    
    **Parks nearby:**
    {parkLocation_findings}       
    
    Your summary should highlight the good coffee shops and childrens park from the two reports.""",
    output_key="todays_event_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


The WorkFlow
============
    
The ParallelAgent runs all its sub-agents simultaneously, the Coffee agent and the park agent.

The Sequential Agent will then submit the outputs of the Parallel Agents and feed it as 
input to the Aggregator agent to summarize the results.


In [9]:

# The ParallelAgent runs all its sub-agents simultaneously.
parallel_Agent = ParallelAgent(
    name="ParallelAgent",
    sub_agents=[coffee_agent, park_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="DayOrganizerSystem",
    sub_agents=[parallel_Agent, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")



✅ Parallel and Sequential Agents created.


Session Management
==================

Set up Session Management. InMemorySessionService stores conversations in RAM (temporary).
It will help store the history of events and retrieve it when needed.

In [10]:

session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: My_Day_Planner
   - User: sami
   - Using: InMemorySessionService


Test the Trip planner 
=====================

Run the agent and see if it can retrieve the data that it already found. Run a conversation 
with couple of queries in the same session


In [11]:
await run_session(
    runner,
    [
        "Hi, I am Samina! Plan my day by finding me a good coffee shop and a nice childrens park halfway between 4260 Albany Dr, San Jose, CA and Red Hawk Terrace, Fremont, CA.",         
        "Hello! What is my name?",  # This time, the agent should remember!     
        "What were the coffee shops you found earlier"
    ],
    "stateful-agentic-session",    
)


 ### Session: stateful-agentic-session

User > Hi, I am Samina! Plan my day by finding me a good coffee shop and a nice childrens park halfway between 4260 Albany Dr, San Jose, CA and Red Hawk Terrace, Fremont, CA.


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


gemini-2.5-flash >  Here are two places for your day plan, located roughly halfway between 4260 Albany Dr, San Jose, CA and Red Hawk Terrace, Fremont, CA:

**Coffee Shop:**
*   **Tamper Room Cafe & Gallery**
*   **Address:** 43737 Boscell Rd, Fremont, CA 94538
*   **Rating:** 4.6 stars

**Children's Park:**
*   **Magical Bridge Playground**
*   **Address:** 540 N Fair Oaks Ave, Sunnyvale, CA 94085
*   **Rating:** 4.6 stars
gemini-2.5-flash >  Samina, I've found a good coffee shop and a nice children's park for your day, both located roughly halfway between your specified addresses:

**Coffee Shop:**
*   **Tamper Room Cafe & Gallery**
*   **Address:** 43737 Boscell Rd, Fremont, CA 94538, United States
*   **Rating:** 4.6 stars

**Children's Park:**
*   **Magical Bridge Playground**
*   **Address:** 540 N Fair Oaks Ave, Sunnyvale, CA 94085, United States
*   **Rating:** 4.6 stars
gemini-2.5-flash >  Samina, I've planned your day with a great coffee shop and a wonderful children's park, b

gemini-2.5-flash >  Your name is Samina.


gemini-2.5-flash >  Your name is Samina.
gemini-2.5-flash >  Your name is Samina.

User > What were the coffee shops you found earlier


gemini-2.5-flash >  The coffee shop I found earlier is:

*   **Tamper Room Cafe & Gallery**
*   **Address:** 43737 Boscell Rd, Fremont, CA 94538, United States
*   **Rating:** 4.6 stars
gemini-2.5-flash >  I found **Tamper Room Cafe & Gallery**.
gemini-2.5-flash >  I found **Tamper Room Cafe & Gallery**, located at 43737 Boscell Rd, Fremont, CA 94538, with a rating of 4.6 stars.
